# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [29]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [30]:
data = pd.read_csv('../data/customer_product_sales.csv').set_index('CustomerID')

In [31]:
data.head()

,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
CustomerID,,,,,,
61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [32]:
data_step_01 = data.groupby(['CustomerID', 'ProductName'])[['Quantity']].sum().reset_index()
data_step_01.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [33]:
data_step_02 = pd.pivot_table(data, index=['ProductName','CustomerID'], values= 'Quantity',fill_value=0)
data_step_02.tail(5)

Quantity
ProductName       CustomerID          
Zucchini - Yellow 94910             25
                  95078             25
                  95372             25
                  96024             25
                  96666             25

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [34]:
data_step_03 = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
distances = squareform(pdist(data_step_03.T, 'euclidean'))
#distances

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [35]:
distancias = pd.DataFrame(1/(1+distances), index=data_step_03.columns, columns=data_step_03.columns)
distancias.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [36]:
distancias.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

Obtenemos la lista de los Customers y elegimos uno. En nuestro caso el 477


In [37]:
customer_477_dist = distancias[477].sort_values(ascending=False)

Descartamos el mismo número porque siempre va a dar que es el más similar.  
Y obtenemos los 5 primeros `[1:6]`.  
Para obtener los elementos tenemos que obtener los índices `.index`.  
Después lo convertimos a un array con el método `to_list()`

In [38]:
top_5 = customer_477_dist[1:6].index.to_list()
top_5

[3535, 3317, 1072, 1008, 639]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [39]:
step_01_df = data_step_01[data_step_01['CustomerID'].isin(top_5)]
step_01_list_top_5= step_01_df['CustomerID'].unique()
step_01_list_top_5

array([ 639, 1008, 1072, 3317, 3535])

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [40]:
ordered_quantity = step_01_df.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
ordered_quantity.head()

,Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [41]:
not_my_products = data_step_03[data_step_03[477]==0]
merged = not_my_products.merge(ordered_quantity, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
merged[1:6].index

Index(['Olive - Spread Tapenade', 'Sprouts - Baby Pea Tendrils',
       'Wine - Blue Nun Qualitatswein', 'Pepper - Black, Whole',
       'Veal - Inside, Choice'],
      dtype='object', name='ProductName')

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [42]:
recommendations_for_everyone = {}
keys = data.index.to_list()
keys
for key in keys:
    cust_dist = distancias[key].sort_values(ascending=False)
    top_5_cust = cust_dist[1:6].index.to_list()
    step_01_new = data_step_01[data_step_01['CustomerID'].isin(top_5_cust)]
    ord_quant = step_01_new.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
    not_my_products = data_step_03[data_step_03[key]==0]
    merged = not_my_products.merge(ord_quant, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    values = merged[1:6].index
    recommendations_for_everyone.update({key: values})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [43]:
results = pd.DataFrame.from_dict(recommendations_for_everyone, orient='index').reset_index()
results.columns = ['CustomerID','Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results.head()

,CustomerID,Prod_01,Prod_02,Prod_03,Prod_04,Prod_05
0,61288,Jagermeister,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural",Flavouring - Orange
1,77352,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt,Guinea Fowl
2,40094,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale,"Oregano - Dry, Rubbed"
3,23548,Pernod,Wanton Wrap,Bread - Raisin Walnut Oval,Banana Turning,Sprouts - Baby Pea Tendrils
4,78981,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda,Beef - Inside Round


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [44]:
data_step_03_cosine = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
dist_cosine = squareform(pdist(data_step_03_cosine.T, 'cosine'))
distancias_cosine = pd.DataFrame(1/(1+dist_cosine), index=data_step_03_cosine.columns, columns=data_step_03_cosine.columns)
recommendations_for_everyone_cosine = {}
keys = data.index.to_list()
keys
for key in keys:
    cust_dist = distancias_cosine[key].sort_values(ascending=False)
    top_5_cust = cust_dist[1:6].index.to_list()
    step_01_new = data_step_01[data_step_01['CustomerID'].isin(top_5_cust)]
    ord_quant = step_01_new.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
    not_my_products = data_step_03_cosine[data_step_03_cosine[key]==0]
    merged = not_my_products.merge(ord_quant, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    values = merged[1:6].index
    recommendations_for_everyone_cosine.update({key: values})

In [45]:
results_cosine = pd.DataFrame.from_dict(recommendations_for_everyone_cosine, orient='index')
results_cosine.columns = ['Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results_cosine.head()

,Prod_01,Prod_02,Prod_03,Prod_04,Prod_05
61288,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji,Snapple Lemon Tea
77352,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney,"Chicken - Leg, Boneless"
40094,Beef - Inside Round,Truffle Cups - Brown,"Bread - Roll, Soft White Round",Cake - Mini Cheesecake,Anchovy Paste - 56 G Tube
23548,Juice - Orange,Veal - Sweetbread,Olives - Stuffed,Cinnamon Buns Sticky,Barramundi
78981,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted,"Soup - Campbells, Beef Barley"


Me sorprende bastante la variación de la recomendación de productos en función de distancia que usemos. Si observamos el head de los datos, aparentemente no existe mucha relación en los customers...
Y por ello, decidimos seguir comprobando con otro tipo de distancia: `jaccard`.

In [46]:
data_step_03_jaccard = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
dist_jaccard = squareform(pdist(data_step_03_jaccard.T, 'jaccard'))
distancias_jaccard = pd.DataFrame(1/(1+dist_jaccard), index=data_step_03_jaccard.columns, columns=data_step_03_jaccard.columns)
recommendations_for_everyone_jaccard = {}
keys = data.index.to_list()
keys
for key in keys:
    cust_dist = distancias_jaccard[key].sort_values(ascending=False)
    top_5_cust = cust_dist[1:6].index.to_list()
    step_01_new = data_step_01[data_step_01['CustomerID'].isin(top_5_cust)]
    ord_quant = step_01_new.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
    not_my_products = data_step_03_jaccard[data_step_03_jaccard[key]==0]
    merged = not_my_products.merge(ord_quant, left_on='ProductName', right_on='ProductName').sort_values('Quantity',ascending=False)
    values = merged[1:6].index
    recommendations_for_everyone_jaccard.update({key: values})

In [47]:
results_jaccard = pd.DataFrame.from_dict(recommendations_for_everyone_jaccard, orient='index')
results_jaccard.columns = ['Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results_jaccard.head()

,Prod_01,Prod_02,Prod_03,Prod_04,Prod_05
61288,Flavouring - Orange,V8 - Berry Blend,Ecolab - Mikroklene 4/4 L,General Purpose Trigger,Cornflakes
77352,Sea Bass - Whole,Beer - Blue,Cattail Hearts,Mayonnaise - Individual Pkg,Beef Ground Medium
40094,Rosemary - Dry,"Juice - Cranberry, 341 Ml","Wine - Red, Colio Cabernet",Pasta - Angel Hair,Snapple - Iced Tea Peach
23548,Table Cloth 81x81 White,Longos - Chicken Wings,Potatoes - Idaho 100 Count,"Lemonade - Natural, 591 Ml","Ice - Clear, 300 Lb For Carving"
78981,Wine - Blue Nun Qualitatswein,Pie Filling - Cherry,Cocoa Butter,Cheese - Mix,Beer - Sleemans Cream Ale
